In [1]:
import pandas as pd

df = pd.read_csv('file/health_result_label2.csv')

# 질문 리스트
questions = df['question'].tolist()
answers = df['answer'].tolist()
df.head()

,question,answer,category02,label
0,"너무 목이 마른데요, 혹시 정수기 같은 건 없나요?",정수기는 진료 대기실 앞 쪽에 있습니다. 감사합니다.,29.대기실 및 진료실 위치 안내,0
1,교정기가 떨어진 것 같아서 붙이러 왔어요.,접수 먼저 부탁드립니다. 카운터에서 성함을 입력해 주시길 바랍니다.,27.진료 접수 안내,1
2,앞으로 얼마나 나아지는지 보고 아프면 오라고 하네요. 제가 다음에 예약하고 와도 되죠?,예약하실 때 지난 번 진료 기록이 있다고 말씀해 주시길 부탁 드립니다. 감사합니다.,33.다음 진료실 예약,2
3,이것도 수납용 키오스크인가요? 대기 줄이 너무 길어서요.,이곳에서도 수납 진행 가능합니다. 홈버튼을 누른 후 수납 버튼을 누르십시오.,31.수납 방법 안내,3
4,주사실 안에 가서 있으라는데 주사실이 어딘데요?,침을 맞는 곳이라 이름이 주사실입니다. 3층으로 올라가시면 됩니다.,29.대기실 및 진료실 위치 안내,0


In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# fine-tuning된 모델과 토크나이저 로드
model = AutoModelForSequenceClassification.from_pretrained("./fine_tuned_model_previous")
tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_model_previous")

In [3]:
import torch
import torch.nn.functional as F
import numpy as np

# numpy의 _reconstruct 함수를 허용 목록에 추가
torch.serialization.add_safe_globals([np.core.multiarray._reconstruct])

# 신뢰할 수 있는 데이터 파일을 로드
question_embeddings = torch.load('embeddings/previous_question_embeddings.pth')

# 평가 모드로 설정
model.eval()

def get_embedding(input_question, tokenizer, model):
    # 입력 문장을 토크나이즈
    inputs = tokenizer(input_question, return_tensors="pt", padding=True, truncation=True)
    
    with torch.no_grad():
        # hidden states를 포함하도록 설정
        outputs = model(**inputs, output_hidden_states=True)
        
        # 마지막 hidden state에서 [CLS] 토큰의 임베딩을 가져옴
        cls_embedding = outputs.hidden_states[-1][:, 0, :]  # [CLS] 토큰의 임베딩
        
    return cls_embedding.squeeze().numpy()  # numpy 배열로 반환



# 코사인 유사도를 계산하여 가장 유사한 답변을 찾는 함수
def find_most_similar_answer_cosine(input_question, question_embeddings, answers, tokenizer, model):
    # 입력 질문 임베딩 생성
    input_embedding = get_embedding(input_question, tokenizer, model)

    max_similarity = -1
    best_answer = None
    
    # 각 질문 임베딩과 유사도 비교
    for i, question_embedding in enumerate(question_embeddings):
        # question_embedding을 텐서로 변환하고 차원 맞추기
        question_embedding_tensor = torch.tensor(question_embedding).unsqueeze(0)  # (1, 768)
        
        # input_embedding도 텐서로 변환하고 차원 맞추기
        input_embedding_tensor = torch.tensor(input_embedding).unsqueeze(0)  # (1, 768)
        
        # 코사인 유사도 계산
        similarity = F.cosine_similarity(input_embedding_tensor, question_embedding_tensor).item()
        
        if similarity > max_similarity:
            max_similarity = similarity
            best_answer = answers[i]

    return best_answer, max_similarity  # 유사도 반환 추가

# 챗봇 응답 함수
def chatbot_response(input_question, tokenizer, model, question_embeddings, answers):
    # 1차 필터링: 분류 모델로 레이블 예측
    inputs = tokenizer(input_question, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_label = torch.argmax(logits, dim=-1).item()

    # 2차 필터링: 같은 카테고리 내에서 코사인 유사도 계산
    # 같은 레이블의 질문들과 임베딩 필터링
    filtered_df = df[df['label'] == predicted_label]
    filtered_indices = filtered_df.index.tolist()

    # 필터링된 질문에 해당하는 미리 계산된 임베딩과 답변 가져오기
    filtered_question_embeddings = [question_embeddings[i] for i in filtered_indices]
    filtered_answers = [answers[i] for i in filtered_indices]

    # 코사인 유사도를 통해 가장 유사한 답변 찾기
    best_answer, cosine_similarity = find_most_similar_answer_cosine(input_question, filtered_question_embeddings, filtered_answers, tokenizer, model)
    
    return best_answer, cosine_similarity, predicted_label  # 세 가지 값 반환


C:\Users\tlsek\AppData\Local\Temp\ipykernel_9356\2153087946.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  question_embeddings = torch.load('embeddings/previous_questio

In [4]:
# 예시 질문
input_question = "예약은 이미 전화로 끝냈습니다. 안 도와줘도 괜찮아요."

# 챗봇 응답 호출
best_answer, cosine_similarity, predicted_label = chatbot_response(input_question, tokenizer, model, question_embeddings, answers)

# 결과 출력

print("예측된 레이블:", predicted_label)
print("최고 유사도 답변:", best_answer)
print("코사인 유사도:", cosine_similarity)

예측된 레이블: 2
최고 유사도 답변: 죄송하지만 접수 마감시간인 6시 30분까지는 방문해 주셔야 합니다.
코사인 유사도: 1.000000238418579


In [5]:
# 예시 질문
input_question = "너무 목이 마른데요, 혹시 정수기 같은 건 없나요?"

# 챗봇 응답 호출
best_answer, cosine_similarity, predicted_label = chatbot_response(input_question, tokenizer, model, question_embeddings, answers)

# 결과 출력

print("예측된 레이블:", predicted_label)
print("최고 유사도 답변:", best_answer)
print("코사인 유사도:", cosine_similarity)



예측된 레이블: 0
최고 유사도 답변: 정수기는 진료 대기실 앞 쪽에 있습니다. 감사합니다.
코사인 유사도: 1.0


In [6]:
# test_questions 데이터프레임 읽기
test_questions = pd.read_csv('result csv/test_questions_with_labels.csv')

# 모든 질문을 리스트로 변환하여 input_questions에 저장
input_questions = test_questions['question'].tolist()
input_labels = test_questions['label'].tolist()  # 라벨 리스트 추가

# 결과를 저장할 리스트 초기화
results = []

# 각 질문에 대해 챗봇 응답 호출
for input_question, input_label in zip(input_questions, input_labels):
    # chatbot_response 함수 호출, 반환 값이 올바른지 확인
    best_answer, cosine_similarity, predicted_label = chatbot_response(
        input_question, tokenizer, model, question_embeddings, answers
    )
    
    # 결과를 리스트에 추가
    results.append({
        "question": input_question,
        "label": input_label,  # 원래 라벨 추가
        "predicted_label": predicted_label,  # 예측된 라벨 추가
        "best_answer": best_answer,
        "cosine_similarity": cosine_similarity
    })

# 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(results)

# 결과 출력
results_df[:30]


,question,label,predicted_label,best_answer,cosine_similarity
0,화상연고도 처방이 되나요?,1,0,우측 복도 끝에 위치해 있습니다. 심리평가실까지 안내해 드리겠습니다.,0.999350
1,오늘 두시 삼십분에 예약했어요. 언제 들어가나요?,1,1,긴급상황이라면 응급실로 이동하셔서 진료 보실 수 있습니다. 응급실 건물로 가주시면 ...,1.000000
2,예약한 시간보다 조금 늦었어요. 다시 접수해야 하나요?,1,1,반갑습니다. 1층 통합 창구에서 번호표를 뽑고 기다려 주시면 안내 도와드리겠습니다.,1.000000
3,"피가 계속 나는데, 저 죽는 걸까요?",1,1,1층 창구에서 진료 접수를 할 수 있습니다. 저를 따라오시면 안내해 드리겠습니다.,1.000000
4,접수 재등록해도 되나요?,1,1,현재 고객님 앞의 대기자가 0명이기 때문에 즉시 진료 보실 수 있습니다.,1.000000
5,접수 시간이 지났는데 혹시 대기순번이 넘어갔나요?,1,1,바로 접수를 도와드리나 대기시간이 있습니다.,1.000000
6,접수가 왜이렇게 오래 걸려요?,1,1,"초진이신 경우, 우선 문진표 작성을 요청드립니다.",1.000000
7,"다음에 들어가는 순번인데, 잠깐 화장실 다녀와도 되나요?",1,0,진료실은 카운터 옆 원장실로 들어가면 됩니다. 로비에서 대기해 주세요.,0.999998
8,접수하려고요.,1,1,예약했어도 접수를 해 주셔야 합니다. 앞에 있는 접수증을 작성해 주시길 부탁드립니다.,1.000000
9,진료 접수 어떻게 하죠?,1,1,네 현재는 현장접수만 하고 있습니다. 빠른 시일 내에 앱에 병원 정보 등록을 요청하...,1.000000


In [7]:
# 정확도 계산
correct_predictions = (results_df['label'] == results_df['predicted_label']).sum()  # 올바른 예측 수
total_predictions = results_df.shape[0]  # 총 예측 수
accuracy = correct_predictions / total_predictions  # 정확도 계산

# 정확도 출력
print(f'정확도: {accuracy * 100:.2f}%')

정확도: 85.92%


In [8]:
# 결과를 CSV 파일로 저장
results_df.to_csv('result csv/chatbot_responses_bayesian_previous.csv', index=False)
